In [ ]:
import os
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import NASNetLarge
from keras.applications.resnet_v2 import ResNet152V2

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix

import pandas as pd
import numpy as np
from glob import glob

import warnings  
warnings.filterwarnings('ignore')

In [ ]:
entrada = '../input/leishmaniose-lamina/'
out = "./"
batch_size = 8
base_entrada = glob(entrada)[0] # Pega a string do nome da pasta

In [ ]:
models = [InceptionV3(weights='imagenet',include_top=True), Xception(weights='imagenet',include_top=True), InceptionResNetV2(weights='imagenet',include_top=True),
          NASNetLarge(weights='imagenet',include_top=True), ResNet152V2(weights='imagenet',include_top=True)]

In [ ]:
data = ImageDataGenerator(rescale = 1./255) # reescala dos dados
# , rotation_range = 7, horizontal_flip = True, shear_range = 0.2, height_shift_range = 0.07, zoom_range = 0.2
for model in models:
    data_gen = data.flow_from_directory(base_entrada, target_size=(model.input.shape[1], model.input.shape[2]), batch_size=batch_size)

    print('Executando model {}'.format(model.name))
    model.layers.pop()
    name_model = model.name # nome do modelo
    model = Model(inputs=model.input,outputs=model.layers[-1].output)
    features = model.predict(data_gen)

    save = np.concatenate([features, np.reshape(data_gen.labels,(-1,1))], axis=1)
    np.savetxt('leishmaniose_{}.csv'.format(name_model), save, delimiter=',')

print(data_gen.class_indices)

In [ ]:
base_saida = glob(os.path.join(out, "*.csv"))

In [ ]:
from IPython.display import Image
Image('../input/confusao/ConfusionMatrix.jpg')

In [ ]:
# Calculo de métricas
def calculo_metricas(y_true, y_pred):    
    
    # tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # print(confusion_matrix(y_true, y_pred))
    acuracia = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1score = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    
    metricas = list(np.round([acuracia, precision, recall, f1score, kappa], 4))
    
    return metricas

In [ ]:
# Treinando com o Classificador Random Forest
from sklearn.neural_network import MLPClassifier

def classificador_mlp(X_treino, Y_treino, X_teste, Y_teste):
    
    print("--------------- MLP ----------------")
    classificador_v1 = MLPClassifier(random_state=1, hidden_layer_sizes=(100,100), max_iter=200)
    classificador_v1.fit(X_treino, Y_treino)
    
    # Verificando nos dados de teste
    predict_teste_v1 = classificador_v1.predict(X_teste)
    print("Teste: ", calculo_metricas(Y_teste, predict_teste_v1))
    
    # Cross-Validation
    cross_validation = cross_val_score(classificador_v1, X, Y, scoring='accuracy', cv=10, n_jobs = -1)
    print("Cross-validation: ", list(np.round(cross_validation, 4)))
    print("Média CV: {0:.4f}".format(cross_validation.mean()))
    print("STD CV: {0:.4f}".format(cross_validation.std()))

In [ ]:
# Classificação Random Forest
print("Métricas Y_teste: [Acurácia, Precision, Recall, F1-score, Kappa]\n\n")

for base in base_saida:
    print("Base: ", base)
    
    # Leitura da base
    dataset = pd.read_csv(base, header=None)
    
    # Divisão entre treino e teste
    X = dataset.iloc[:, 0:-1]
    Y = dataset.iloc[:,-1]
    X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = 0.2, random_state = 1, stratify=Y)
    
    # MLP
    classificador_mlp(X_treino, Y_treino, X_teste, Y_teste)
    
    print("\n\n")

In [ ]:
Y_treino.value_counts()

In [ ]:
Y_teste.value_counts()